# Simple calculation with basic projection formulas (with/without the use of proj)

Figuring out the differences created by using the true scale latitude in an equirectangular projection. The same approach could be used e.g. to text angles, or areas, with two or more points, as needed.

## Equirectangular projection equations

From [Snyder](https://pubs.er.usgs.gov/publication/pp1395) (1987), see also [Wikipedia on Equirectangular projection page](https://en.wikipedia.org/wiki/Equirectangular_projection)

## Symbols

From Snyder (1987): 

<div class="alert alert-block alert-info">  
    
$x =$ rectangular coordinate: distance to the right of the vertical line (Y axis)  passing through the origin or center of a projection (if negative, it is distance to the left). In practice, a "false" x or "false easting'' is frequently added to all values of x to eliminate negative numbers

$y =$ rectangular coordinate: distance above the horizontal line (X axis) passing through the origin or center of a projection (if negative, it is distance below). In practice, a "false" y or "false northing" is frequently added to all values of y to eliminate negative numbers.

$h =$ relative scale factor along a meridian of longitude. (For general perspective projections, h is height above surface of ellipsoid.)

$k =$ relative scale factor along a parallel of latitude

$\lambda = $ longitude east

$\lambda _{0} =$ longitude east of the central meridian of the map, or of the origin of the rectangular coordinates (for west longitude, use a minus sign). 

$\phi = $ north geodetic or geographic latitude (if latitude is south, apply a minus sign).

$\phi _{1} =$ single standard parallel on cylindrical or conic projections; latitude of central point on azimuthal projections.
    
$R =$ radius of the sphere, either actual or that corresponding to scale of the map.
</div>


--- 


--- 

### Direct formulas (x,y metric from lat, lon degrees)

Snyder's (1987) equations 12-1 to 12-4:

$ x = R (\lambda - \lambda _{0}) cos \phi _{1}$

$y = R \phi$

$h = 1$

$k = cos \phi _{1} / cos \phi$

### Inverse formulas (lat, lon degrees, from x, y metric)

Snyder's (1987) equations 12-5, 12-6

$\phi = y / R$

$\lambda = \lambda _{0} + \frac{x}{R cos \phi _{1}}$

## Use case, check Jezero with and without "latitude of true scale" set

Jezero is centered at:

**Longitude = 77.58 E**

**Latitude = 18.38 N**

THE Mars2020 team uses this:
    
```
 +proj=eqc +lat_ts=18.4663 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +R=3396190 +units=m +no_defs
```

Note ```+lat_ts=18.4663```

Our GIS data will use this:
    
```
+proj=eqc +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +R=3396190 +units=m +no_defs
```

i.e. ```+lat_ts=0``` if not specified (Please see: https://proj.org/operations/projections/eqc.html).



### Check the location of 2 points form the official traverse

see also https://mars.nasa.gov/maps/?mission=M20

e.g. 

<!-- ![overview](./images/overview_155_477.jpg)
 -->
 ![overview](https://raw.githubusercontent.com/europlanet-gmap/winter-school-2023/main/crs/notebooks/images/overview_155_477.jpg)
 
#### Sol 155 - First Drill site

<!-- ![155](./images/mars2020_sol155_zoom.jpg)
 -->
![155](https://raw.githubusercontent.com/europlanet-gmap/winter-school-2023/main/crs/notebooks/images/mars2020_sol155_zoom.jpg)

```
{"longitude":{"value":77.4515774846077,"unit":"degree"},"latitude":{"value":18.427776796857273,"unit":"degree"},"easting":{"value":4590918.828103036,"unit":"meter"},"northing":{"value":1092300.8956607156,"unit":"meter"}}
```

Latitude = 18.427776796857273

Longitude = 77.4515774846077

Eastings (eqc) = 4590918.828103036

Northings (eqc) = 1092300.8956607156

#### Sol 477 

<!-- ![477](./images/mars2020_sol477_zoom.jpg)
 -->
 ![155](https://raw.githubusercontent.com/europlanet-gmap/winter-school-2023/main/crs/notebooks/images/mars2020_sol477_zoom.jpg)


Latitude = 18.458887871256312

Longitude = 77.40616417917408

Eastings (eqc) = 4588226.968159924

Northings (eqc) = 1094144.9951853438

```
{"longitude":{"value":77.40616417917408,"unit":"degree"},"latitude":{"value":18.458887871256312,"unit":"degree"},"easting":{"value":4588226.968159924,"unit":"meter"},"northing":{"value":1094144.9951853438,"unit":"meter"}}
```

--- 

We can do some rounding (many decimal digits!)

### Calculate x, y in CRS with and without _+lat_ts_

One can do this with [cs2cs](https://proj.org/apps/cs2cs.html) as CLI of proj, or e.g. with [pyproj](https://pyproj4.github.io/pyproj/stable/api/transformer.html). But since the formulas for the Equirectangular/Equidistant Cylindrical over a sphere are not complex, one can exemplify it by hand.


In [2]:
from math import pi

lon_sol_155_deg = 77.451577 # 6 decimal digits, i.e. ~decimetric 
lat_sol_155_deg = 18.427777
lon_sol_477_deg = 77.406164
lat_sol_477_deg = 18.458888

# they need to be in radians
# radian = degree*(pi/180)

lon_sol_155 = lon_sol_155_deg*(pi/180)
lat_sol_155 = lat_sol_155_deg*(pi/180)
lon_sol_477 = lon_sol_477_deg*(pi/180)
lat_sol_477 = lat_sol_477_deg*(pi/180)


# map projection parameters for Jezero 
lon_0 = 0 # central longitude
lat_0 = 0 # central latitude
x_0 = 0 # false eastings
y_0 = 0 # false northings 
R = 3396190 # major radius of IAU2000 Mars ellipsoid A=3396190 m B=3376200 m

lat_ts = 0 # default latitiude of true scale at equator

# additional parameter for Mars2020 team:

lat_ts_mars2020_deg = 18.4663
lat_ts_mars2020 = lat_ts_mars2020_deg*(pi/180)


### Calculate eastings, northings of points in both map projections

In [3]:
from math import cos

# Direct formulas for equirectangular
def direct_eqc(lon, lat, lat_ts):
    eastings_eqc = R*(lon - lon_0)*cos(lat_ts)
    northings_eqc = R*lat
    return(eastings_eqc, northings_eqc)

# Inverse formulas for equirectangular
def inverse_eqc(eastings, northings, lat_ts):
    lon = northings/R
    lat = lon_0 + eastings/(R*cos(lat_ts))
    return(lon, lat)

In [4]:
# calculate for Sol 155 & lat_ts 0
eastings_155_eqc, northings_155_eqc = (direct_eqc(lon_sol_155, lat_sol_155, 0))
print(eastings_155_eqc, northings_155_eqc)

4590918.799378062 1092300.9077019393


In [5]:
# calculate for Sol 155 & lat_ts = 18.4663
eastings_155_team, northings_155_team = (direct_eqc(lon_sol_155, lat_sol_155, lat_ts_mars2020))
print(eastings_155_team, northings_155_team)

4354532.951082128 1092300.9077019393


In [6]:
# calculate for Sol 477 & lat_ts = 0
eastings_477_eqc, northings_477_eqc = (direct_eqc(lon_sol_477, lat_sol_477, 0))
print(eastings_477_eqc, northings_477_eqc)

4588226.9575394355 1094145.002816587


In [7]:
# calculate for Sol 477 & lat_ts = 18.4663
eastings_477_team, northings_477_team = (direct_eqc(lon_sol_477, lat_sol_477, lat_ts_mars2020))
print(eastings_477_team, northings_477_team)

4351979.711851021 1094145.002816587


## Check with pyproj (this could have saved some time ;-) )

see also https://pyproj4.github.io/pyproj/stable/api/transformer.html

In [8]:
from pyproj import Transformer

# CRS from proj4 code
lonlat_mars_sphere = "+proj=latlong +R=3396190 +no_defs"
eqc_proj = "+proj=eqc +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +R=3396190 +units=m +no_defs"
mars2020_team_proj = "+proj=eqc +lat_ts=18.4663 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +R=3396190 +units=m +no_defs"

# set transformatiomn
lonlat_to_eqc = Transformer.from_crs(lonlat_mars_sphere, eqc_proj)
lonlat_to_mars2020team = Transformer.from_crs(lonlat_mars_sphere, mars2020_team_proj)

print('latitude, longitude for Sol 155 are:')
print(lon_sol_155_deg, lat_sol_155_deg, '\n')
      
print('Eastings, northigns for Sol 155 in Equirectangular lat_ts=0 are:')
print(lonlat_to_eqc.transform(lon_sol_155_deg, lat_sol_155_deg))

print('Eastings, northigns for Sol 155 in Equirectangular lat_ts=18.4663 (Mars2020 Team) are:')
print(lonlat_to_mars2020team.transform(lon_sol_155_deg, lat_sol_155_deg),'\n')

print('Eastings northings for Sol 155 calculated ~by hand in Equirectangular lat_ts=0 are:')
print(eastings_155_eqc, northings_155_eqc)

print('Eastings northings for Sol 155 calculated ~by hand in Equirectangular lat_ts=18.4663 (Mars2020 Team) are:')
print(eastings_155_team, northings_155_team)
print('------------------------------------\n')

print('latitude, longitude for Sol 477 are:')
print(lon_sol_477_deg, lat_sol_477_deg, '\n')
      
print('Eastings, northigns for Sol 477 in Equirectangular lat_ts=0 are:')
print(lonlat_to_eqc.transform(lon_sol_477_deg, lat_sol_477_deg))

print('Eastings, northigns for Sol 477 in Equirectangular lat_ts=18.4663 (Mars2020 Team) are:')
print(lonlat_to_mars2020team.transform(lon_sol_477_deg, lat_sol_477_deg),'\n')

print('Eastings northings for Sol 477 calculated ~by hand in Equirectangular lat_ts=0 are:')
print(eastings_477_eqc, northings_477_eqc)

print('Eastings northings for Sol 477 calculated ~by hand in Equirectangular lat_ts=18.4663 (Mars2020 Team) are:')
print(eastings_477_team, northings_477_team)
print('------------------------------------\n')

# lonlat_to_eqc

latitude, longitude for Sol 155 are:
77.451577 18.427777 

Eastings, northigns for Sol 155 in Equirectangular lat_ts=0 are:
(4590918.799378062, 1092300.9077019393)
Eastings, northigns for Sol 155 in Equirectangular lat_ts=18.4663 (Mars2020 Team) are:
(4354532.951082128, 1092300.9077019393) 

Eastings northings for Sol 155 calculated ~by hand in Equirectangular lat_ts=0 are:
4590918.799378062 1092300.9077019393
Eastings northings for Sol 155 calculated ~by hand in Equirectangular lat_ts=18.4663 (Mars2020 Team) are:
4354532.951082128 1092300.9077019393
------------------------------------

latitude, longitude for Sol 477 are:
77.406164 18.458888 

Eastings, northigns for Sol 477 in Equirectangular lat_ts=0 are:
(4588226.9575394355, 1094145.002816587)
Eastings, northigns for Sol 477 in Equirectangular lat_ts=18.4663 (Mars2020 Team) are:
(4351979.71185102, 1094145.002816587) 

Eastings northings for Sol 477 calculated ~by hand in Equirectangular lat_ts=0 are:
4588226.9575394355 1094145.002

Comparable with:
    
(from MMGIS)

**Sol 155**

{"longitude":{"value":77.4515774846077,"unit":"degree"},"latitude":{"value":18.427776796857273,"unit":"degree"},"easting":{"value":4590918.828103036,"unit":"meter"},"northing":{"value":1092300.8956607156,"unit":"meter"}}

**Sol 477**

{"longitude":{"value":77.40616417917408,"unit":"degree"},"latitude":{"value":18.458887871256312,"unit":"degree"},"easting":{"value":4588226.968159924,"unit":"meter"},"northing":{"value":1094144.9951853438,"unit":"meter"}}

### Calculate distance among points for the 2 pairs of x,y

In [9]:
from math import sqrt

delta_x_eqc = eastings_477_eqc - eastings_155_eqc
delta_y_eqc = northings_477_eqc - northings_155_eqc

delta_x_team = eastings_477_team - eastings_155_team
delta_y_team = northings_477_team - northings_155_team

# distance among the locations of Sol 155 and Sol 477
d_eqc = sqrt(delta_x_eqc**2 + delta_y_eqc**2)
d_team = sqrt(delta_x_team**2 + delta_y_team**2)

print('distance between position at Sol 155 and 477 (Equirectangular, lat_ts=0) = ', d_eqc)
print('distance between position at Sol 155 and 477 (TEAM, lat_ts=18.4663) = ', d_team)

distance between position at Sol 155 and 477 (Equirectangular, lat_ts=0) =  3262.9280218916474
distance between position at Sol 155 and 477 (TEAM, lat_ts=18.4663) =  3149.5582806371876


In [10]:
delta_d = d_eqc - d_team
print('the difference in distance measured between a lat_ts=0 and lat_ts centered in Jezero is ', delta_d, ' over ', d_team)

difference = delta_d/d_team*100
print('the difference is in', difference,'%')

the difference in distance measured between a lat_ts=0 and lat_ts centered in Jezero is  113.36974125445977  over  3149.5582806371876
the difference is in 3.5995441631111498 %


# References

PROJ contributors (2020). PROJ coordinate transformation software library. Open Source Geospatial Foundation. URL https://proj.org/. DOI: 10.5281/zenodo.5884394

Snow, A., Whitaker, et al. (2022). pyproj4/pyproj: 3.4.1 Release (3.4.1). Zenodo. https://doi.org/10.5281/zenodo.7430570

Snyder (1987) Map projections: A working manual, USGS Professional Paper No. 1395, 385 p. - https://pubs.er.usgs.gov/publication/pp1395 / https://doi.org/10.3133/pp1395